In [1]:
import sys
sys.path.insert(0, "../")
import dolphindb as ddb 
import numpy as np
from factor_cal.feature import features as fe
from factor_cal.factor import factors as fa
from factor_cal.config_loader import basic_config as cfg
from factor_cal.utils import ddb_utils as du
from factor_cal.utils import tools as tl
from factor_cal.factor import factor_func as ff

# read config file
config = cfg.BasicConfig('config/config.yml')
# obtain the ddb session
s = du.DDBSessionSingleton().session

In [2]:
features = fe.Features(config)
factors = fa.Factors(config, features)

Loading [feature]close from DolphinDB server
Loading [feature]vol from DolphinDB server


In [7]:
factors.fac_dict['close_ret'].data

AttributeError: 'BasicFactor' object has no attribute 'data'

In [3]:
features.get_feature('open').get_secs()

Loading [feature]open from DolphinDB server


array(['000002', '000004'], dtype=object)

In [5]:
factors.process()

RuntimeError: Only DolphinDB Table object is accepted

In [8]:
tl.show_memory("before")
data = features.get_feature('open')
tl.show_memory("after")

before 一共占用97.74MB
Loading [feature]open from DolphinDB server
after 一共占用98.07MB


In [4]:
data2 = features.get_feature('en_buy_n')
tl.show_memory("after")

Loading [feature]en_buy_n from DolphinDB server
after 一共占用82.39MB


In [8]:
data3 = features.get_feature('close')
tl.show_memory("after")

after 一共占用83.28MB


In [10]:
type(data)

numpy.ndarray

In [5]:
def ret(x:np.ndarray, shift:int=1) -> np.ndarray:
    tl.show_memory("1")
    x_shift = np.roll(x, shift, axis=0)
    tl.show_memory("2")
    x_shift[0:shift] = np.nan
    tl.show_memory("3")
    return x / x_shift - 1

In [9]:
tl.show_memory("before")

before 一共占用98.07MB


In [11]:
ret(data, 2)
tl.show_memory("after")

1 一共占用100.37MB
2 一共占用100.88MB
3 一共占用101.15MB


In [1]:
class Test:
    def __init__(self, config):
        self.config = config

In [2]:
config = {"haha": 1, "xixi": 2}
a = Test(config)

In [7]:
from factor_cal.factor.factor_func import rolling_correlation

In [9]:
import numpy as np
# 示例数据
X = np.random.rand(20, 3)
Y = np.random.rand(20, 3)
window_size = 5

# 计算滚动相关系数
result = rolling_correlation(X, Y, window_size)
print(result)

[[        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]
 [-0.06415948 -0.77119084  0.47933507]
 [-0.1388156  -0.56483018  0.7894413 ]
 [ 0.39608284 -0.24158028  0.68497714]
 [ 0.28345174  0.08111292  0.6417023 ]
 [ 0.1897983   0.08529828 -0.10257587]
 [-0.37552798  0.73213619 -0.70018968]
 [-0.30786228  0.3865624  -0.83415388]
 [ 0.02849382 -0.10486703 -0.80734297]
 [ 0.45480929 -0.4341194  -0.29579527]
 [ 0.51408755 -0.44365239 -0.17112812]
 [ 0.66978688 -0.49163883  0.0794473 ]
 [ 0.81410937  0.22983714  0.25877051]
 [ 0.60351261  0.84221541  0.30902703]
 [ 0.44510123  0.90098583  0.29449704]
 [ 0.32868554  0.84844318  0.38109441]
 [ 0.4564897   0.63053386 -0.54008907]]


In [18]:
xc, yc = X[:5, 0], Y[:5, 0]
xc = [1, 2, 3, 4, 5]
yc = [5, 4, 3, 2, 1]
# np.correlate(X[:5, 0], Y[:5, 0]), np.corrcoef(X[:5, 0], Y[:5, 0])

from scipy.stats import pearsonr, spearmanr
pearsonr(xc, yc), 
type(spearmanr(xc, yc)[0])

numpy.float64

In [13]:
t = s.loadTable(tableName='t_trade_info')
a = t.exec('open').pivotby('trade_time', 'security_code')

In [17]:
b = a.toDF()

In [ ]:
tmp_feat = config['features'][0]
ddb_name = tmp_feat['ddb_name']
tb_feat = tmp_feat['tb_features']

tb_name = 'trade_info'
time_col = tb_feat[tb_name]['time_col']
sec_col = tb_feat[tb_name]['sec_col']

In [ ]:
tables = fe.DDB_FeatTable(ddb_name, tb_name, time_col, sec_col)

In [ ]:
tables.get_table(config['start_time'], config['end_time'], ('000002', '000004'))

In [ ]:
a = {"a": 2}
c = a.get('b', None)
c is None

In [ ]:
s.run('select * from t_trade_info')

In [ ]:
ddb_name, tb_feat

In [ ]:
print(ddb_name)
# create a dictionary to store features
index = 0
feat_dict = {}
for tb in tb_feat.keys():
    print("Table Name: ", tb)
    ddb_tb = s.loadTable(dbPath=ddb_name, tableName=tb)
    
    features = tb_feat[tb]['feat_cols']
    sec_col = tb_feat[tb]['sec_col']
    time_col = tb_feat[tb]['time_col']
    print("Features contain: ", list(features.keys()))

    # read features from DolphinDB in a panel format
    for feat_nkname, feat_colname in features.items():
        index += 1
        if (index > 1):
            break
        print("Feature Name: ", feat_nkname, flush=True)
        feat_df = ddb_tb.select(feat_colname).pivotby(time_col, sec_col).toDF()
        feat_dict[feat_nkname] = feat_df
        # print(feat_df.head(1), flush=True)


In [ ]:
print(ddb_name)
# create a dictionary to store features
index = 0
feat_dict = {}
for tb in tb_feat.keys():
    print("Table Name: ", tb)
    ddb_tb = s.loadTable(dbPath=ddb_name, tableName=tb)
    
    features = tb_feat[tb]['feat_cols']
    sec_col = tb_feat[tb]['sec_col']
    time_col = tb_feat[tb]['time_col']
    print("Features contain: ", list(features.keys()))

    # read features from DolphinDB in a panel format
    for feat_nkname, feat_colname in features.items():
        index += 1
        if (index > 2):
            break
        print("Feature Name: ", feat_nkname, flush=True)
        feat_df = ddb_tb.select(feat_colname).pivotby(time_col, sec_col).toDF()
        feat_dict[feat_nkname] = feat_df
        # print(feat_df.head(1), flush=True)


In [ ]:
tb = 'trade_info'
s.run(f'select close from loadTable("{ddb_name}", "{tb}") pivot by {time_col}, {sec_col}')

In [ ]:
# import sys
# print(sys.getsizeof(feat_dict['open']) / 1024) # KB
# print(sys.getsizeof(feat_dict['open']) / 1024 / 1024) # MB
# print(sys.getsizeof(feat_dict['open']) / 1024 / 1024 / 1024) # GB

In [ ]:
f'select close from loadTable({ddb_name}, {tb}) pivot by {time_col}, {sec_col}'

In [ ]:
ddb_name, tb

In [ ]:
a = s.loadTable(dbPath=ddb_name, tableName=tb)

In [ ]:
b = a.select('close').where('trade_time < 2023-04-20T09:30:00.000')

In [ ]:
trade = s.loadTableBySQL(tableName="trade_info", dbPath="dfs://CROSS_SECTION_6sec", sql="select * from trade_info where second(trade_time)<09:50:00", tableAliasName='t1')


In [ ]:
trade

In [ ]:
import ddb_utils as du

In [ ]:
s = du.DDBSessionSingleton().session

In [ ]:
table = s.run('select * from loadTable("dfs://CROSS_SECTION_6sec", "trade_info") where second(trade_time) between 09:30:06 and 09:30:18 limit 10')

In [ ]:
a = s.loadTable("trade_info", "dfs://CROSS_SECTION_6sec")
b = a.select("*").where("second(trade_time) between 09:30:06 and 09:30:18").showSQL()

In [ ]:
s.run('tmp_b = ' + b)

In [ ]:
s.run('objs(true)')